In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
import cv2

In [13]:

train_f = "/content/train"
test_f = "/content/test"
valid_f = "/content/validation"
mask_train = os.path.join(train_f,'mask')
nomask_train = os.path.join(train_f,'nonmask')

In [14]:
mask_train_name = os.listdir(mask_train)
print(mask_train_name[:25])
nomask_train_name = os.listdir(nomask_train)
print(nomask_train_name[:25])

['0006.jpg', '0029.jpg', '0040.jpg', '0043.jpg', '0037.jpg', '0003.jpg', '0018.jpg', '0019.jpg', '0042.jpg', '0022.jpg', '0048.jpg', '0027.jpg']
['13.jpg', '1.jpg', '0.jpg', '10.jpg', '6.jpg', '12.jpg', '11.jpg', '2.jpg', '3.jpg', '7.jpg', '4.jpg', '9.jpg']


In [15]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   zoom_range = 0.3,
                                   rotation_range = 35,
                                   horizontal_flip = True
                                   )
test_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_f,
                                                    target_size=(150,150),
                                                    batch_size = 32,
                                                    class_mode = 'binary'
                                                    )
test_generator = test_datagen.flow_from_directory(test_f,
                                                    target_size=(150,150),
                                                    batch_size = 32,
                                                    class_mode = 'binary'
                                                    )
valid_generator = validation_datagen.flow_from_directory(valid_f,
                                                    target_size=(150,150),
                                                    batch_size = 32,
                                                    class_mode = 'binary'
                                                    )

Found 24 images belonging to 3 classes.
Found 24 images belonging to 3 classes.
Found 24 images belonging to 3 classes.


In [16]:
train_generator.class_indices

{'.ipynb_checkpoints': 0, 'mask': 1, 'nonmask': 2}

In [17]:
train_generator.image_shape

(150, 150, 3)

In [29]:
from tensorflow.keras.applications import vgg16

 

# Init the VGG model

vgg_conv = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(600, 600, 3))

In [30]:
# Freeze all the layers

for layer in vgg_conv.layers[:]:
    layer.trainable = False

 

# Check the trainable status of the individual layers

for layer in vgg_conv.layers:
    print(layer, layer.trainable)


<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7f2e17895c10> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f2e2256c490> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f2e178a0c10> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f2e1788d250> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f2e01d4fb10> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f2e01d86a50> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f2e178a6750> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f2e178bc4d0> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f2e178fa890> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f2e178fa950> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f2e178a4a50> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 

In [31]:
# Create the model

model = Sequential()

 

# Add the vgg convolutional base model

model.add(vgg_conv)

 

# Add new layers

model.add(Flatten())

model.add(Dense(1024, activation='relu'))

model.add(Dropout(0.5))

model.add(Dense(3, activation='softmax'))

 

# Show a summary of the model. Check the number of trainable parameters

model.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 18, 18, 512)       14714688  
_________________________________________________________________
flatten_2 (Flatten)          (None, 165888)            0         
_________________________________________________________________
dense_5 (Dense)              (None, 1024)              169870336 
_________________________________________________________________
dropout_4 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 3)                 3075      
Total params: 184,588,099
Trainable params: 169,873,411
Non-trainable params: 14,714,688
_________________________________________________________________


In [32]:
# Load the normalized images

train_datagen = ImageDataGenerator(rescale=1./255)

validation_datagen = ImageDataGenerator(rescale=1./255)

 

# Change the batchsize according to your system RAM

train_batchsize = 100

val_batchsize = 10

 

# Data generator for training data

train_generator = train_datagen.flow_from_directory(
        "/content/train",
        target_size=(600, 600),
        batch_size=10,
        class_mode='categorical')

 

# Data generator for validation data

validation_generator = validation_datagen.flow_from_directory(
        "/content/validation",
        target_size=(600, 600),
        batch_size=10,
        class_mode='categorical',
        shuffle=False)


Found 24 images belonging to 3 classes.
Found 24 images belonging to 3 classes.


In [33]:
# Configure the model for training

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

 

# Train the model

history = model.fit(
      train_generator,
      steps_per_epoch=
         train_generator.samples/train_generator.batch_size,
      epochs=20,
      validation_data=validation_generator,
      validation_steps=
         validation_generator.samples/validation_generator.batch_size,
      verbose=1)


Epoch 1/20
2/2 [==============================] - 179s 114s/step - loss: 14.0793 - acc: 0.5417 - val_loss: 12.0510 - val_acc: 0.5000
Epoch 2/20
2/2 [==============================] - 175s 98s/step - loss: 10.0013 - acc: 0.5542 - val_loss: 10.5487 - val_acc: 0.5000
Epoch 3/20
2/2 [==============================] - 182s 99s/step - loss: 3.9267 - acc: 0.7500 - val_loss: 10.7845 - val_acc: 0.5000
Epoch 4/20
2/2 [==============================] - 175s 98s/step - loss: 4.6030 - acc: 0.6607 - val_loss: 3.7188 - val_acc: 0.5417
Epoch 5/20
2/2 [==============================] - 175s 99s/step - loss: 0.2358 - acc: 0.9375 - val_loss: 3.7852 - val_acc: 0.5000
Epoch 6/20
2/2 [==============================] - 179s 116s/step - loss: 1.3335 - acc: 0.7976 - val_loss: 1.1405 - val_acc: 0.7083
Epoch 7/20
2/2 [==============================] - 178s 116s/step - loss: 0.1143 - acc: 0.9613 - val_loss: 0.1056 - val_acc: 0.9583
Epoch 8/20
2/2 [==============================] - 174s 98s/step - loss: 1.2808e-04